# 1. Thông tin chung
Thành viên:

|MSSV|Họ tên|
|----|-----|
|19120080|Lê Đức Huy|
|19120298|Mai Duy Nam|

# 2. Thông tin về dữ liệu

## 2.1 Thông tin chung
Dataset trong đồ án này được lấy từ Kaggle tại [đây](https://www.kaggle.com/halhuynh/it-jobs-dataset).

Dataset này chứa thông tin về các công ty IT ở Việt Nam và các công việc được đăng tải bởi các công ty này. Dữ liệu này được tác giả crawl từ trang web [ITviec](https://itviec.com/). Dataset này bao gồm 2 file:
* `companies.csv` chứa thông tin về các công ty (tên công ty, lượt rating, lượt review, địa chỉ, v.v.) có tăng tải việc làm trên ITviec
* `jobs.csv` chứa thông tin về việc làm được đăng tải bởi các công ty (tên công việc, mô tả, yêu cầu kỹ năng, v.v.)

Hai tập dữ liệu này có mối liên hệ với nhau thông qua cột `company_id`.

## 2.2 Khám phá dữ liệu

Ở các phần sau, ta lần lượt khám phá 2 bộ dữ liệu `companies` và `jobs` thu thập được

### 2.2.1 `companies`

#### Nạp dữ liệu

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
companies_df = pd.read_csv('dataset/companies.csv')
companies_df.head()

,company_id,company_name,average_rating,num_review,city,type,num_employee,country,working_day,OT,overview,expertise,benifit,logo_link
0,kms-technology,KMS Technology,4.0,197 Reviews,"Ho Chi Minh, Da Nang, Ha Noi",Outsourcing,1000+,United States,Monday - Friday,No OT,"Established in 2009, KMS Technology is a U.S.-...",NaN,Want to be the Best? Join KMS!\n\n\nIt’s posit...,https://itviec.com/rails/active_storage/repres...
1,nab-in-collaboration-with-positive-thinking-co...,NAB in collaboration with Positive Thinking Co...,4.4,11 Reviews,Ho Chi Minh,Product,501-1000,Australia,Monday - Friday,No OT,NAB in collaboration with Positive Thinking Co...,Languages &amp; Frameworks\n\n\nJavaScript ES6...,THE BENEFITS AND PERKS\n\n\n\n\nVery competiti...,https://itviec.com/rails/active_storage/repres...
2,robert-bosch-engineering-and-business-solutions,Robert Bosch Engineering And Business Solutions,3.2,217 Reviews,"Ho Chi Minh, Ha Noi",Product,1-50,Germany,Monday - Friday,No OT,Robert BOSCH Engineering and Business Solution...,Embedded Software for Automotive Applications\...,Why \nBOSCH\n?\n\nBecause we don't just follow...,https://itviec.com/rails/active_storage/repres...
3,gft-technologies-vietnam,GFT Technologies Vietnam,NaN,Reviews,"Ho Chi Minh, Ha Noi",Outsourcing,51-150,Germany,Monday - Friday,No OT,Working at GFT Vietnam\n\n\nGFT is driving the...,"Technical frameworks\n\n\nJava/Kotlin, Spring ...",HR benefits\n\n\n\n\nCompetitive salary\n\n\nS...,https://itviec.com/rails/active_storage/repres...
4,tyme,Tyme,4.6,29 Reviews,"Ho Chi Minh, Ha Noi",Product,151-300,Vietnam,Monday - Friday,No OT,Tyme Vietnam\n is the international arm of \nT...,Mobile/ Tablet Development: \nNative Android/i...,Our goal is simple: to be the #1 digital techn...,https://itviec.com/rails/active_storage/repres...


#### Số lượng dòng và cột

In [3]:
print('Number of rows: {}\nNumber of columns: {}'.format(*companies_df.shape))

Number of rows: 2041
Number of columns: 14


#### Các dòng có ý nghĩa gì?
Mỗi dòng chứa thông tin của một công ty dựa theo trang ITviec. Mỗi công ty có một ID duy nhất.

Kiểm tra có dòng nào bị lặp hay không.

In [4]:
companies_df.duplicated().sum() != 0

False

Kiểm tra có ID công ty nào bị lặp hay không.

In [5]:
companies_df.duplicated('company_id').sum() != 0

False

#### Các cột có ý nghĩa gì?
Tác giả của bộ dữ liệu không mô tả chi tiết về ý nghĩa của các một mà chỉ nói "The column names are self-explained". Do đó ta phải tự suy luận về ý nghĩa của các cột:
* `company_id`: ID của công ty
* `company_name`: tên công ty
* `average_rating`: số điểm rating trung bình của công ty trên thang điểm 5
* `num_review`: số lượt review công ty
* `city`: lhành phố
* `type`: loại hình công ty (outsourcing hay product)
* `num_employee`: số lượng nhân viên
* `country`: quốc gia
* `working_day`: ngày làm việc trong tuần
* `OT`: các chính sách về làm thêm giờ
* `overview`: tổng quan về công ty
* `expertise`: lĩnh vực chuyên ngành
* `benifit` (sai chính tả, phải là `benefit`): lợi ích khi tham gia công ty
* `logo_link`: đường dẫn đến logo công ty

Có một vấn đề về ý nghĩa các cột: cột `city` không thống nhất với cột `country` (có công ty có thành phố là `Ho Chi Minh` nhưng quốc gia là `Australia`). Do các công ty có thể có nhiều trụ sở đặt ở nhiều quốc gia, trong cùng quốc gia cũng được đặt ở nhiều thành phố khác nhau, cho nên trong các bước phân tích tiếp theo ta sẽ giả sử hai cột `city` và `country` có ý nghĩa như sau:
* Cột `city` chứa các thành phố ở Việt Nam nơi các văn phòng của công ty được đặt.
* Cột `country` chứa quốc gia nơi trụ sở chính của công ty được đặt.

Các cột còn lại đều có ý nghĩa rõ ràng, không có gì gây khó hiểu.

Trước khi đến bước phân tích tiếp theo, ta thực hiện một số thay đổi về các cột như sau:
* Đổi tên các cột cho hợp lý:
    * Đổi tên cột `benifit` bị sai chính tả
    * Đổi tên cột `city`, `num_review`, `num_employee` và `working_day` thành số nhiều
    * Đổi tên cột `OT` thành `overtime` để thống nhất với các tên cột còn lại
* Loại bỏ cột `logo_link` do không cần thiết

In [6]:
companies_df.rename(columns={'benifit': 'benefit',
                             'city': 'cities',
                             'num_review': 'num_reviews',
                             'num_employee': 'num_employees',
                             'working_day': 'working_days',
                             'OT': 'overtime'}, inplace=True)
companies_df.drop(columns='logo_link', inplace=True)

#### Mỗi cột có kiểu dữ liệu thế nào và có phù hợp để xử lý tiếp không?

In [7]:
companies_df.dtypes

company_id         object
company_name       object
average_rating    float64
num_reviews        object
cities             object
type               object
num_employees      object
country            object
working_days       object
overtime           object
overview           object
expertise          object
benefit            object
dtype: object

Có hai cột có thể phải cân nhắc chuyển về dạng số là `num_reviews` và `num_employees`. Các cột khác đều đã có kiểu dữ liệu phù hợp.

Đối với cột `num_employees`, dữ liệu của ta được thu thập theo dạng một khoảng giá trị. Việc chuyển đổi cột này về dạng số hay không phụ thuộc vào phân tích nào mà ta đang thực hiện. Do đó tạm thời ở bước này ta không thay đổi cột `num_employees`.

In [8]:
companies_df.num_employees.unique()

array(['1000+', '501-1000', '1-50', '51-150', '151-300', '301-500'],
      dtype=object)

Đối với `num_reviews`, kiểm tra các giá trị unique của nó, ta thấy các giá trị đều có dạng: `"<một số> Review[s]"`, ngoại trừ giá trị duy nhất không tuân theo pattern này là `Reviews`. Vì vậy, ta có thể lấy ra giá trị số, lọc bỏ chuỗi ` Reviews` dư thừa và chuyển kiểu dữ liệu cột thành dạng float. Đối với giá trị `Reviews`, ta gán bằng NaN.

In [9]:
companies_df.num_reviews.unique()

array(['197 Reviews', '11 Reviews', '217 Reviews', 'Reviews',
       '29 Reviews', '16 Reviews', '8 Reviews', '60 Reviews', '9 Reviews',
       '127 Reviews', '6 Reviews', '34 Reviews', '3 Reviews',
       '20 Reviews', '185 Reviews', '1486 Reviews', '57 Reviews',
       '22 Reviews', '68 Reviews', '1 Review', '21 Reviews', '12 Reviews',
       '23 Reviews', '7 Reviews', '17 Reviews', '5 Reviews', '96 Reviews',
       '37 Reviews', '207 Reviews', '109 Reviews', '4 Reviews',
       '33 Reviews', '76 Reviews', '18 Reviews', '10 Reviews',
       '2 Reviews', '65 Reviews', '15 Reviews', '41 Reviews',
       '19 Reviews', '27 Reviews', '13 Reviews', '26 Reviews',
       '42 Reviews', '183 Reviews', '24 Reviews', '54 Reviews',
       '14 Reviews', '38 Reviews', '451 Reviews', '28 Reviews',
       '36 Reviews', '52 Reviews', '31 Reviews', '92 Reviews',
       '30 Reviews', '86 Reviews', '32 Reviews', '25 Reviews',
       '43 Reviews', '46 Reviews', '97 Reviews', '44 Reviews',
       '226 Revi

In [10]:
import re
def get_num_reviews(x):
    result = re.match('^(\d+) Reviews?$', x)
    return float(result.group(1)) if result != None else np.nan
companies_df.num_reviews = companies_df.num_reviews.map(get_num_reviews, na_action='ignore')
companies_df.num_reviews.unique()

array([1.970e+02, 1.100e+01, 2.170e+02,       nan, 2.900e+01, 1.600e+01,
       8.000e+00, 6.000e+01, 9.000e+00, 1.270e+02, 6.000e+00, 3.400e+01,
       3.000e+00, 2.000e+01, 1.850e+02, 1.486e+03, 5.700e+01, 2.200e+01,
       6.800e+01, 1.000e+00, 2.100e+01, 1.200e+01, 2.300e+01, 7.000e+00,
       1.700e+01, 5.000e+00, 9.600e+01, 3.700e+01, 2.070e+02, 1.090e+02,
       4.000e+00, 3.300e+01, 7.600e+01, 1.800e+01, 1.000e+01, 2.000e+00,
       6.500e+01, 1.500e+01, 4.100e+01, 1.900e+01, 2.700e+01, 1.300e+01,
       2.600e+01, 4.200e+01, 1.830e+02, 2.400e+01, 5.400e+01, 1.400e+01,
       3.800e+01, 4.510e+02, 2.800e+01, 3.600e+01, 5.200e+01, 3.100e+01,
       9.200e+01, 3.000e+01, 8.600e+01, 3.200e+01, 2.500e+01, 4.300e+01,
       4.600e+01, 9.700e+01, 4.400e+01, 2.260e+02, 1.790e+02, 6.600e+01,
       4.500e+01, 4.900e+01, 5.800e+01, 2.320e+02, 8.700e+01])

#### Với mỗi cột kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [11]:
numerical_columns = ['average_rating', 'num_reviews']
categorical_columns = companies_df.columns.to_list()
categorical_columns.remove('average_rating')
categorical_columns.remove('num_reviews')

In [12]:
companies_df[numerical_columns].describe()

,average_rating,num_reviews
count,563.000000,869.000000
mean,3.987034,12.989643
std,0.597378,57.202636
min,2.000000,1.000000
25%,3.600000,2.000000
50%,4.000000,4.000000
75%,4.400000,10.000000
max,5.000000,1486.000000


In [13]:
companies_df[numerical_columns].isna().sum() / companies_df[numerical_columns].shape[0] * 100

average_rating    72.415483
num_reviews       57.422832
dtype: float64

Có thể thấy, số lượng giá trị thiếu của các cột số là khá lớn. Còn lại, các số liệu thống kê khác đều ổn, không có điều gì bất thường.

#### Với mỗi cột dạng phân loại, các giá trị được phân bố thế nào

In [14]:
desc = companies_df[categorical_columns].describe()
desc

,company_id,company_name,cities,type,num_employees,country,working_days,overtime,overview,expertise,benefit
count,2041,2041,2041,2041,2041,2041,2026,1865,2040,477,1576
unique,2041,2041,71,2,6,39,2,4,2022,474,1573
top,alfa-connections,Toyo System & Nikkoku Vietnam,"District 1, Ho Chi Minh",Product,1-50,Vietnam,Monday - Friday,No OT,.,Agile methodology,Very attractive salaries that commensurate wit...
freq,1,1,190,1534,1054,1202,1855,1589,12,2,2


Tỷ lệ giá trị thiếu

In [15]:
(1 - desc.loc['count'] / companies_df.shape[0]) * 100

company_id             0.0
company_name           0.0
cities                 0.0
type                   0.0
num_employees          0.0
country                0.0
working_days      0.734934
overtime          8.623224
overview          0.048996
expertise        76.629103
benefit           22.78295
Name: count, dtype: object

Một số giá trị ngẫu nhiên

In [16]:
companies_df[categorical_columns].sample(8)

,company_id,company_name,cities,type,num_employees,country,working_days,overtime,overview,expertise,benefit
635,fiingroup,FiinGroup,"Ha Noi, Ho Chi Minh",Product,51-150,Vietnam,Monday - Friday,Extra salary for OT,is a pioneer and now a Vietnam’s leader in fin...,- Customer Centric- Works are all fun- Simple ...,"Become our staff, you will get experience with..."
1418,jamalex,JAMALEX,"Binh Thanh, Ho Chi Minh",Product,51-150,Singapore,Monday - Friday,No OT,Về Jamalex Finan\n\n\nJamalex Finan là Công ty...,NaN,"Chế độ lương, thưởng, quyền mua cổ phiếu... hấ..."
352,kas,KAS,"Thu Duc City, Ho Chi Minh",Product,51-150,Vietnam,Monday - Saturday,No OT,KAS là đơn vị phát triển và cung cấp giải pháp...,NaN,"Môi trường làm việc năng động, chuyên nghiệp\n..."
634,ecomobi,Ecomobi,"Dong Da, Ha Noi",Product,151-300,Vietnam,Monday - Friday,NaN,Ecomobi aims to be One-Stop monetization platf...,NaN,100% Salary for Probation.\n\n\nLunch allowanc...
1121,hamsa-technologies,Hamsa Technologies,"Cau Giay, Ha Noi",Outsourcing,51-150,Vietnam,Monday - Friday,Extra salary for OT,Hamsa Technologies is one of leading e-commerc...,React + Redux + React DnD + Storybook + Materi...,"As Hamsa always value the human factor, we wil..."
169,vntravel-group,VNtravel Group,"Ha Noi, Ho Chi Minh",Product,301-500,Vietnam,Monday - Friday,No OT,VNTravel Group is an online traveling market w...,"Infra: Linux, google cloud, AWS, k8s CI/CD: Dr...","Salary, bonus, insurance\n\n\n\n\nSalary: 500$..."
887,k-system-and-solutions,K.System And Solutions,"Ho Chi Minh, Ha Noi",Outsourcing,51-150,"Korea, Republic of",Monday - Friday,No OT,The main business line includles \n-Consultanc...,NaN,"Working place: \n\n\n\n800 Nguyen Van Linh, Th..."
600,hl-solutions-jsc,HL Solutions JSC,"Go Vap, Ho Chi Minh",Outsourcing,1-50,Vietnam,Monday - Friday,No OT,HL Solutions (Happy Life Solutions) là công ty...,NaN,"- Được leader giỏi tận tâm chỉ dạy, lo..."


Có thể thấy dữ liệu dạng phân loại khá đầy đủ (ngoại trừ cột `expertise` số lượng giá trị thiếu lên đến 70%).

# 3. Câu hỏi cần trả lời

## 3.1 Câu hỏi 1

### 3.1.1 Nêu vấn đề

### 3.1.2 Data Preprocessing

### 3.1.3 Analysis, Visualization

### 3.1.4 Kết luận

## 3.2 Câu hỏi 2

### 3.2.1 Nêu vấn đề

### 3.2.2 Data Preprocessing

### 3.2.3 Analysis, Visualization

### 3.2.4 Kết luận

## 3.3 Phương hướng phát triển nếu còn thời gian

# 4. Cảm nhận sau khi kết thúc môn học

## 4.1 Lê Đức Huy - 19120080

## 4.2 Mai Duy Nam - 19120298

# 5. Tài liệu tham khảo